# Load the data

In [ ]:
require 'yaml'
require 'sqlite3'

setlists_yaml = File.read('setlist.yml')
setlists = YAML.load(setlists_yaml)

# Open a database
db = SQLite3::Database.new "songs.db"

nil


# Map the songs to their IDs

In [ ]:
songs = setlists.flat_map { |i| i }.uniq

select_song = <<-SQL
  SELECT
    s.title,
    s.id,
    a.id
  FROM
    songs s
    LEFT JOIN arrangements a on a.song_id = s.id
  WHERE
    s.title = ?
SQL

songs_from_db = songs.map do |song_title|
  result = db.get_first_row(select_song, song_title )
  puts "Missing song: #{song_title}" unless result
  result
end


# Connect to services

In [ ]:
require 'pco_api'

token =  ENV['PCO_API_TOKEN']
secret = ENV['PCO_API_SECRET']

api = PCO::API.new(basic_auth_token: token, basic_auth_secret: secret)

nil

# Get weeks to update

In [ ]:
initial_week = '2021-08-01'
num_weeks = setlists.length

get_services = <<-SQL
  SELECT
    p.id, st.id, p.dates
  FROM
    plans p
    LEFT JOIN service_types st on st.id = p.service_type_id
  WHERE
    st.name = "Sunday Morning Worship"
    AND
    p.dates >= ?
  ORDER BY 
    p.dates asc
  LIMIT ?
SQL

services = db.execute(get_services, initial_week, num_weeks)


# Update Weeks

In [ ]:
services.each_with_index do |service, service_index|
  p service[2]
  
  # Get song plan items
  plan_songs_sql = <<-SQL
  SELECT
    *
  FROM
    plan_items pi
  WHERE
    pi.plan_id = ?
    AND
    pi.item_type = "item"
    AND
    pi.title LIKE 'Song%'
SQL
  
  set = setlists[service_index]
  
  plan_songs = db.execute(plan_songs_sql, service[0])
  
  # Update song plan items
  
  plan_songs.each_with_index do |song, song_index|
    set_song = set[song_index]
    db_song = songs_from_db.find { |song| song[0] == set_song}
    
    if set_song
      puts "\"#{song[2]}\" becomes \"#{set_song}(#{db_song[1]}, #{db_song[2]})\""
      data = { 
        attributes: {
          title: set_song, 
          song_id: db_song[1],
          arrangement_id: db_song[2]
        }
      }
      #api.services.v2.service_types[service[1].to_i].plans[service[0].to_i].items[song[0].to_i].patch(data: data)
    end
    

  end
end

nil